In [ ]:
# goal: pass a signature of a hashed composite number to check

In [10]:
from Crypto.PublicKey import RSA
from Crypto.Hash import MD5
from Crypto.Signature import pkcs1_15
from Crypto.Util.number import long_to_bytes, isPrime

key = RSA.generate(1024)
sig_scheme = pkcs1_15.new(key)
key

RsaKey(n=130536157427813017823418191924632476476487748599325067005927790670320075403104634800778512784217138600329821127645533928869702088657523001600638832303102854001228627311509282391867811884755063377844121263716260439144584305348285109390063390367657654610909159060388417219203772057210729959981058120036202349761, e=65537, d=772815799960197307101122396001608265146058660856281581370828429438091295854320434299740039371290260111509080329073152027121235491388359623129649921931182362182586423162615939243043068442256812157838343523628620150586533622444311179904576560056605541090114385139749962329628138993363187409309278898918740673, p=10756559624246935509024383388952480589111324236501509603138708828332807322377811495807102358869655903931374175145165118310440741945138942346393722477976221, q=12135493316428469750660857326085624513703663566412959108643711555833329280201172881413119861266020925778049863289370588452994798570353915252419101256570741, u=35606276549442908408154985252088198487908

In [32]:
from Crypto.Util.number import bytes_to_long
msg1 = open("message1.bin", 'rb').read()
msg2 = open("message2.bin", 'rb').read()
z = 1
while True:
    prime = bytes_to_long(msg1 + long_to_bytes(z))
    composite = bytes_to_long(msg2 + long_to_bytes(z))
    if isPrime(prime) and not isPrime(composite):
        break
    z += 2
print(sig_scheme.sign(MD5.new(long_to_bytes(prime))))
print(sig_scheme.sign(MD5.new(long_to_bytes(composite))))

b'\xb4r\xcf\x05\xd4? \xf4\xa5)\xd3\xda;\xceDkm\xce\xbdn\x82\xed8\xf7\xd3%\x99\x9f\x8b\x8d%,y/H\x80\x82\x8e\xb0\x8c\xf1\x98_\xefM\xc3@\x07L\x85%kaS\xeaEkw3\x8eO\xf0\x84B\x85~;ZH\x87H\x832VO\xe7\xf7\xd4m\x87\xfe\xfc`\x86\xd9\x02\x03m\xc1\xab\xd7\xb9o\xbf%=]\xb5\xf4\x99c\xbb"|.\xde\x99\xa9S\xba=\x0e\xfc\xe7"\xf6\xe8\xfd\xca\x12?|\x92\xa1K\xd3\x99\xe5'
b'\xb4r\xcf\x05\xd4? \xf4\xa5)\xd3\xda;\xceDkm\xce\xbdn\x82\xed8\xf7\xd3%\x99\x9f\x8b\x8d%,y/H\x80\x82\x8e\xb0\x8c\xf1\x98_\xefM\xc3@\x07L\x85%kaS\xeaEkw3\x8eO\xf0\x84B\x85~;ZH\x87H\x832VO\xe7\xf7\xd4m\x87\xfe\xfc`\x86\xd9\x02\x03m\xc1\xab\xd7\xb9o\xbf%=]\xb5\xf4\x99c\xbb"|.\xde\x99\xa9S\xba=\x0e\xfc\xe7"\xf6\xe8\xfd\xca\x12?|\x92\xa1K\xd3\x99\xe5'


In [45]:
for i in range(1,100):
    if (composite%i==0): print(i)

1
7
11
17
71
77


In [42]:
from pwnlib.tubes.remote import remote
from cryptolib import json_send, json_recv
r = remote("socket.cryptohack.org", 13392)
r.recvuntil(b"Primality checking is expensive so I made a service that signs primes, allowing anyone to quickly check if a number is prime\n")
json_send(r, {"option": "sign", "prime": prime})
signature = json_recv(r)['signature']
json_send(r, {"option": "check", "prime": composite, "signature": signature, "a": 71})
json_recv(r)

{'msg': 'Valid signature. First byte of flag: crypto{MD5_5uck5_p4rt_tw0}'}